In [2]:
!pip install yfinance ta plotly

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=d368ae0dc427a94ff206ba17c27514adbbfd344e445e60330e2f510f5d9fbe41
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [4]:
import yfinance as yf
import pandas as pd
import time
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.momentum import RSIIndicator, StochasticOscillator
from IPython.display import clear_output, display
from zoneinfo import ZoneInfo
from datetime import datetime

# === Config ===
symbol = 'RAYMOND.NS'  # Change to any NSE/BSE symbol
interval = '5m'
period = '5d'
refresh_seconds = 60

# === Init Signal Storage ===
signals = pd.DataFrame(columns=['Time (IST)', 'Signal', 'Price'])

while True:
    try:
        df = yf.download(
            tickers=symbol,
            interval=interval,
            period=period,
            auto_adjust=False,  # Explicit to avoid warning
            progress=False
        )

        if df.empty:
            print("No data found. Retrying...")
            time.sleep(refresh_seconds)
            continue

        df.index = df.index.tz_convert('Asia/Kolkata')

        # ✅ Convert to Series to avoid ndarray error
        close = df['Close'].squeeze()
        high = df['High'].squeeze()
        low = df['Low'].squeeze()
        volume = df['Volume'].squeeze()

        # === Indicators ===
        df['SMA_20'] = SMAIndicator(close, window=20).sma_indicator()
        df['EMA_20'] = EMAIndicator(close, window=20).ema_indicator()
        df['RSI'] = RSIIndicator(close, window=14).rsi()
        macd = MACD(close)
        df['MACD'] = macd.macd()
        df['MACD_signal'] = macd.macd_signal()

        stoch = StochasticOscillator(high=high, low=low, close=close)
        df['Stoch_K'] = stoch.stoch()
        df['Stoch_D'] = stoch.stoch_signal()

        # === Pivot Point Classic ===
        last_close = close.iloc[-2]
        last_high = high.iloc[-2]
        last_low = low.iloc[-2]
        pivot = (last_high + last_low + last_close) / 3
        df['Pivot'] = pivot

        # === Volume Analysis ===
        df['Volume_avg'] = volume.rolling(window=20).mean()

        # === Latest Values ===
        row = df.iloc[-1]
        ts = df.index[-1]
        close_val = row['Close'].item()
        sma = row['SMA_20'].item()
        ema = row['EMA_20'].item()
        rsi = row['RSI'].item()
        macd_val = row['MACD'].item()
        macd_sig = row['MACD_signal'].item()
        stoch_k = row['Stoch_K'].item()
        stoch_d = row['Stoch_D'].item()
        vol = row['Volume'].item()
        vol_avg = row['Volume_avg'].item()

        # === Signal Logic ===
        signal = None
        if (
            close_val > ema and
            rsi > 50 and
            macd_val > macd_sig and
            stoch_k > stoch_d and
            vol > vol_avg
        ):
            signal = "BUY"
        elif (
            close_val < ema and
            rsi < 50 and
            macd_val < macd_sig and
            stoch_k < stoch_d and
            vol < vol_avg
        ):
            signal = "SELL"

        # === Save Signal ===
        if signal:
            signals = pd.concat([signals, pd.DataFrame([{
                'Time (IST)': ts.strftime('%Y-%m-%d %H:%M:%S'),
                'Signal': signal,
                'Price': round(close_val, 2)
            }])], ignore_index=True)

        # === Display ===
        clear_output(wait=True)
        print(f"Last Updated: {datetime.now(ZoneInfo('Asia/Kolkata')).strftime('%Y-%m-%d %H:%M:%S')} IST")
        display(signals.tail(20))

        time.sleep(refresh_seconds)

    except KeyboardInterrupt:
        print("Stopped manually.")
        break
    except Exception as e:
        print(f"Error: {e}")
        time.sleep(refresh_seconds)


Last Updated: 2025-05-13 10:55:52 IST


,Time (IST),Signal,Price


Stopped manually.
